# Evalution of vmango-lab simulations

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import xsimlab as xs
import xsimlab.dot as dot
import ipywidgets as iw
from importlib import resources

import vmlab
from vmlab.models import *

In [ ]:
dot.dot_graph(vmango,
              show_inputs=False,
              rankdir='TB',
              graph_attr = {'margin':'0.2', 'nodesep':'0.1', 'ranksep':'0.1'})

## 1. Run simulations for the three trees (B10, B12 and F2)

In [ ]:
start_date='2003-06-01'
end_date='2005-12-01'

In [ ]:
def create_setup(tree):
    return vmlab.create_setup(
        model=vmango,
        tree=tree,   
        start_date=start_date,
        end_date=end_date,
        setup_toml='vmango.toml',
        current_cycle=3,
        input_vars={
            'topology__seed': 10,
            'geometry__interpretation_freq': 30
        },
        output_vars={
            'topology': {
                'appearance_date': 'day',
                'appearance_month':'day',
                'bursted': 'day',
                'appeared':'day',
                'nb_gu': None,
                'cycle': 'day',
                'adjacency' : None,
                'is_apical' : None
            },
            'phenology':{
                'nb_inflo':'day',
                'nb_fruit':'day',
                'flowered':'day',
                'fruited':'day',
                'full_bloom_date': None,
                'inflo_stage':'day'
            },
            'arch_dev': {
                'pot_burst_date': 'day',
                'pot_flowering_date' : 'day',
                'pot_nature': 'day',
                'pot_has_apical_child': 'day',
                'pot_nb_lateral_children': 'day',
                'pot_nb_inflo': 'day',
                'pot_nb_fruit': 'day'
            },
            'arch_dev_rep': {
                'flowering': 'day',
                'flowering_date' : 'day',
                'flowering_week' : 'day',
                'fruiting':'day',
                'nb_fruits':'day'
            }               
        }
    )

- **Tree B10**

In [ ]:
with resources.path('vmlab.data.trees', 'tree_B10_cycle_030405.csv') as path:
    tree = pd.read_csv(path)

setup = create_setup(tree)

ds_out_B10 = vmlab.run(setup, vmango, geometry = False)

- **Tree B12**

In [ ]:
with resources.path('vmlab.data.trees', 'tree_B12_cycle_030405.csv') as path:
    tree = pd.read_csv(path)

setup = create_setup(tree)

ds_out_B12 = vmlab.run(setup, vmango, geometry = False)

- **Tree F2**

In [ ]:
with resources.path('vmlab.data.trees', 'tree_F2_cycle_030405.csv') as path:
    tree = pd.read_csv(path)

setup = create_setup(tree)

ds_out_F2 = vmlab.run(setup, vmango, geometry = False)

## 2. Evaluation

### 2.1. Vegetative growth

In [ ]:
appearance_date_all = None
for ds_out in [ds_out_B10, ds_out_B12, ds_out_F2]:
    appearance_date = ds_out.topology__appearance_date.where(ds_out.topology__appeared == 1).groupby(ds_out.topology__appearance_date)
    appearance_date = appearance_date.count().to_dataframe().reindex(pd.date_range(start=start_date, end=end_date, freq='MS'))
    if appearance_date_all is None:
        appearance_date_all = appearance_date
    else:
        appearance_date_all = appearance_date_all.add(appearance_date, fill_value=0)
appearance_date_all = appearance_date_all.rename(columns={"topology__appearance_date": "Simulated Data"})
appearance_date = appearance_date_all


In [ ]:
from numpy import datetime64 as datetime
measured_histogram = [((2003, 5), 0), ((2003, 6), 1), ((2003, 7), 1), ((2003, 8), 0), ((2003, 9), 33), ((2003, 10), 240), ((2003, 11), 71), ((2003, 12), 329), ((2004, 1), 126), ((2004, 2), 323), ((2004, 3), 21), ((2004, 4), 61), ((2004, 5), 7), ((2004, 6), 3), ((2004, 7), 0), ((2004, 8), 213), ((2004, 9), 55), ((2004, 10), 31), ((2004, 11), 144), ((2004, 12), 15), ((2005, 1), 85), ((2005, 2), 723), ((2005, 3), 1189), ((2005, 4), 0)]
def todate(y,m):
    return datetime(str(y)+'-'+str(m).zfill(2)+'-01')
from pandas import DataFrame
measured_histogram = DataFrame({'Measured Data' : [v for k,v in measured_histogram]}, index = [todate(*k) for k,v in measured_histogram])
measured_histogram = measured_histogram.reindex(appearance_date.index)


In [ ]:
fig, axes = plt.subplots(figsize=(10,5))
appearance_date.plot.bar( ax=axes)
measured_histogram.plot(ax=axes, color='purple', linewidth=2, use_index=False)
plt.title('Number of new growth units')
plt.show()

**Figure A. Monthly demography of new growth units (GUs) for growing cycles 1 and 2**

Blue histogram and purple line represent respectively the simulations and observed data summed over the three mango trees (B10, B12 and F2)

### 2.2. Flowering

#### Number of new inflorescences at the full bloom stage for cycle 5

In [ ]:
x_range = list(range(30,51,1))
flowering_date = None

for ds_out in [ds_out_B10, ds_out_B12, ds_out_F2]:
    flowering_date_tree = ds_out.arch_dev__pot_flowering_date.where(ds_out.phenology__flowered == 1).groupby(ds_out.arch_dev__pot_flowering_date)
    flowering_date_tree = flowering_date_tree.count().to_dataframe()
    df_week = flowering_date_tree.index.isocalendar()[['year','week']]
    df = flowering_date_tree.join(df_week)
    df = df[df['year'] == 2005].drop(labels=['year'],axis=1).groupby(['week']).sum().reindex(x_range)
    if flowering_date is None:
        flowering_date = df
    else:
        flowering_date = flowering_date.add(df, fill_value=0)
flowering_date = flowering_date.rename(columns={"arch_dev__pot_flowering_date": "Simulated Data"})


In [ ]:
bloom_date2005 = [((2005, 26), 0), ((2005, 27), 0), ((2005, 28), 0),  ((2005, 29), 0),((2005, 30), 0),
             ((2005, 31), 1), ((2005, 32), 22), ((2005, 33), 3), ((2005, 34), 15), ((2005, 35), 108),
             ((2005, 36), 487), ((2005, 37), 161), ((2005, 38), 204), ((2005, 39), 142),
             ((2005, 40), 49), ((2005, 41), 34), ((2005, 42), 11),  ((2005, 43), 7)]
bloom_date2005 = DataFrame({'Measured Data' : [v for k,v in bloom_date2005]}, 
                           index = [w for (y,w),v in bloom_date2005])
bloom_date2005 = bloom_date2005.reindex(flowering_date.index)


In [ ]:
axes = flowering_date.plot.bar(figsize=(10,5))
bloom_date2005.plot.line(ax=axes, color='purple', linewidth=2, use_index=False)
_= plt.title('Number of new inflorescences at the full bloom stage')

### 2.3. Fruiting

#### Number of new fruits in 2004 and 2005 (cycles 2 and 3)

In [ ]:
nb_fruits = None
for ds_out in [ds_out_B10, ds_out_B12, ds_out_F2]:
    nb_fruits_tree = ds_out.arch_dev__pot_nb_fruit.max(dim="day").to_dataframe()
    flowering_date = ds_out.arch_dev__pot_flowering_date.where(ds_out.topology__appeared == 1).to_dataframe().dropna()
    df = nb_fruits_tree.join(flowering_date)
    df = df.loc[df.index.repeat(df.arch_dev__pot_nb_fruit)].groupby(['arch_dev__pot_flowering_date']).count()
    df_year = df.index.isocalendar()[['year']]
    df = df.join(df_year)
    df = df.groupby(['year']).sum()
    if nb_fruits is None:
        nb_fruits = df
    else:
        nb_fruits = nb_fruits.add(df, fill_value=0)

nb_fruits = nb_fruits.rename(columns={"arch_dev__pot_nb_fruit": "Simulated Data"})
nb_fruits


In [ ]:
nbmeasuredfruits = DataFrame({'Measured Data':[302, 400]},index=nb_fruits.index)
nbmeasuredfruits

In [ ]:
fig, axes = plt.subplots(figsize=(5,5))
ax = nb_fruits.plot.bar(ax=axes)
nbmeasuredfruits.plot(style='o',ax=axes,color='purple',  use_index=False)
_ = plt.title('Number of fruits')

### 4. Distribution of axis lengths 

Axis length is expressed in number of successive apical GUs

In [ ]:
from scipy.sparse import csgraph

def compute_axis_lengths(ds):
    adjacency = np.array(ds.topology__adjacency.data)
    
    # Change topology so that node from cycle 3 are isolated (not considered)
    adjacency[:,np.where(ds.topology__cycle[-1] == 3)] = 0
    adjacency[np.where(ds.topology__cycle[-1] == 3),:] = 0
    
    # Remove link between lateral children and their parent
    adjacency[:,np.where(ds.topology__is_apical == False)] = 0
    for gu in ds.GU:
        lat_children = np.where((adjacency[:,gu] == 1)&(ds.topology__is_apical == False))
        adjacency[lat_children,gu] = 0
    
    n_components, labels = csgraph.connected_components(adjacency)
    print(n_components, labels)
    branch_length = np.bincount(labels[np.where(ds.topology__cycle[-1] != 3) ])
    print(branch_length, max(branch_length))
    branch_length_histogram = np.bincount(branch_length)
    print(branch_length_histogram)
    print()
    return DataFrame({'Simulated data':branch_length_histogram},index = list(range(len(branch_length_histogram))))

In [ ]:
lengthdistribution = None
for ds_out in [ds_out_B10, ds_out_B12, ds_out_F2]:
    df = compute_axis_lengths(ds_out)
    if lengthdistribution is None:
        lengthdistribution = df
    else:
        lengthdistribution = lengthdistribution.add(df, fill_value=0)


In [ ]:
measuredlengthdistribution = DataFrame({'Measured data' : [1282, 797, 533, 309, 27, 2]}, index=list(range(1,7)))


In [ ]:
fig, axes = plt.subplots(figsize=(10,5))
lengthdistribution.plot.bar(ax=axes)
measuredlengthdistribution.plot.line(ax=axes, color='purple', linewidth=2, use_index=False)
_= plt.title('Axis length (in number of successive apical GUs)')